In [ ]:
# Install requests-html package to parse HTML

#pip install requests-html
#pip install fake-useragent

In [11]:
# Import Libraries

import requests
from bs4 import BeautifulSoup
import json
import random
import concurrent.futures
import time
import pandas as pd

from fake_useragent import UserAgent

from datetime import datetime

Please select the World Cup Edition for which you want to scrape data from the list below. In the following example, ID 41087 corresponds to the 2022 World Cup, while ID 15586 corresponds to the 2018 World Cup. Simply click on one of the links below to find the equivalent for another World Cup edition.

In [12]:
# World Cup 2018 ID: 41087
# World Cup 2022 ID: 15586

id = str(input())

41087


In [14]:
url = "https://api.sofascore.com/api/v1/unique-tournament/16/season/"+id+"/standings/total"

# Store collected data at your OS Temporary Directory (/tmp)
ua = UserAgent()

# Use 'ua.ie' or 'ua.random' to get User-Agent

# user-agent = "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36"
payload = ""
headers = {
    "authority": "api.sofascore.com",
    "user-agent": ua.random,
    "accept": "*/*",
    "accept-language": "en-GB,en-US;q=0.9,en;q=0.8,es;q=0.7",
    "cache-control": "max-age=0",
    "if-none-match": "W/^\^f67c1b8fb4^^",
    "origin": "https://www.sofascore.com",
    "referer": "https://www.sofascore.com/",
    "sec-ch-ua": "^\^Google"
}

response_standings = requests.request("GET", url, data=payload, headers=headers)

json_standings = response_standings.json()

standings_dict = {'Group': [], 'Team': [], 'Position': []}

for i in range(len(json_standings['standings'])):
    for j in range(len(json_standings['standings'][i]['rows'])):
        standings_dict['Group'].append(json_standings['standings'][i]['name'])
        standings_dict['Team'].append(json_standings['standings'][i]['rows'][j]['team']['name'])
        standings_dict['Position'].append(json_standings['standings'][i]['rows'][j]['position'])

phase1_df = pd.DataFrame(standings_dict)

phase1_df

Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "C:\Users\chris\anaconda3\lib\site-packages\fake_useragent\utils.py", line 154, in load
    for item in get_browsers(verify_ssl=verify_ssl):
  File "C:\Users\chris\anaconda3\lib\site-packages\fake_useragent\utils.py", line 99, in get_browsers
    html = html.split('<table class="w3-table-all notranslate">')[1]
IndexError: list index out of range


FakeUserAgentError: Maximum amount of retries reached

The URLs that use the ID of the World Cup Edition that you chose at the start. Please keep in mind that the ID used throughout the links corresponds to the World Cup Edition selected at the start.

In [63]:
import requests

# URLs to connect to the API for Round of 16, QF, SF, 3rd Place & Final
url_r16 = "https://api.sofascore.com/api/v1/unique-tournament/16/season/"+id+"/events/round/4/slug/18"
url_q4 = "https://api.sofascore.com/api/v1/unique-tournament/16/season/"+id+"/events/round/3/slug/quarterfinals"
url_s2 = "https://api.sofascore.com/api/v1/unique-tournament/16/season/"+id+"/events/round/2/slug/semifinals"
url_3p = "https://api.sofascore.com/api/v1/unique-tournament/16/season/"+id+"/events/round/50/slug/match-for-3rd-place"
url_f1 = "https://api.sofascore.com/api/v1/unique-tournament/16/season/"+id+"/events/round/1/slug/final"
second_phase_url = [url_r16, url_q4, url_s2, url_3p, url_f1]

def response(url):
    """
    Function to get the response request from HTML. Parameter: URL of the API. Output: Response request from HTML.
    """

    # Store collected data at your OS Temporary Directory (/tmp)
    ua = UserAgent()

    payload = ""
    headers = {
        "authority": "api.sofascore.com",
        "user-agent": ua.random,
        "accept": "*/*",
        "accept-language": "en-GB,en-US;q=0.9,en;q=0.8,es;q=0.7",
        "cache-control": "max-age=0",
        "origin": "https://www.sofascore.com",
        "referer": "https://www.sofascore.com/",
        "sec-ch-ua": "^\^Google"
    }

    response = requests.request("GET", url, data=payload, headers=headers)

    return response

# Define Dictionary to Append the Scores
scores_dict = {'Phase': [], 
               'HomeTeam': [], 'AwayTeam': [], 
               'HomeNormalTimeScore': [], 'AwayNormalTimeScore': [], 
               'HomeOvertimeScore': [], 'AwayOvertimeScore': [],
               'HomePenaltiesScore': [], 'AwayPenaltiesScore': []}

def scores(url):
    """
    Function to get the scores from the API connection to the URL. Parameter: URL of the API. Output: Scores dictionary based on URL.
    """

    response_scores = response(url)

    if response_scores.ok:
        json_scores = response_scores.json()

        for i in range(len(json_scores['events'])):
            scores_dict['Phase'].append(json_scores['events'][i]['roundInfo']['name'])
            scores_dict['HomeTeam'].append(json_scores['events'][i]['homeTeam']['name'])
            scores_dict['AwayTeam'].append(json_scores['events'][i]['awayTeam']['name'])
            scores_dict['HomeNormalTimeScore'].append(int(json_scores['events'][i]['homeScore']['normaltime']))
            scores_dict['AwayNormalTimeScore'].append(int(json_scores['events'][i]['awayScore']['normaltime']))
            try:
                scores_dict['HomeOvertimeScore'].append(int(json_scores['events'][i]['homeScore']['overtime']))
                scores_dict['AwayOvertimeScore'].append(int(json_scores['events'][i]['awayScore']['overtime']))
            except:
                scores_dict['HomeOvertimeScore'].append(0)
                scores_dict['AwayOvertimeScore'].append(0)       
            try:
                scores_dict['HomePenaltiesScore'].append(int(json_scores['events'][i]['homeScore']['penalties']))
                scores_dict['AwayPenaltiesScore'].append(int(json_scores['events'][i]['awayScore']['penalties']))
            except:
                scores_dict['HomePenaltiesScore'].append(0)
                scores_dict['AwayPenaltiesScore'].append(0)

        return scores_dict

    else:
        return scores_dict

# Initialize Scores DataFrame
phase2_df = pd.DataFrame()

# Request Only when Second Phase of Tournament Starts
r16_string_input = "03/12/2022"
r16_start = datetime.strptime(r16_string_input, "%d/%m/%Y")
present = datetime.now()

# Ensures Start of Round of 16 before Getting HTML Requests
if r16_start.date() <= present.date():
    # Append the Scores Dictionary if Response is OK, Otherwise Return the Latest Appended Dictionary
    for i in second_phase_url:
        scores(i)

    phase2_df = pd.DataFrame(scores_dict)

    phase2_df['HomeTotalScore'] = phase2_df['HomeNormalTimeScore'] + phase2_df['HomeOvertimeScore'] + phase2_df['HomePenaltiesScore']
    phase2_df['AwayTotalScore'] = phase2_df['AwayNormalTimeScore'] + phase2_df['AwayOvertimeScore'] + phase2_df['AwayPenaltiesScore']
    phase2_df['TeamQualified'] = phase2_df.apply(lambda x: x['HomeTeam'] if (x['HomeTotalScore'] > x['AwayTotalScore']) else x['AwayTeam'], axis = 1)

else:
    pass

# Second Phase Dataframe
phase2_df